In [26]:
import pandas
from sklearn.metrics import cohen_kappa_score
from  nltk.metrics import agreement
import numpy as np
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [43]:
df1 = pandas.read_csv('../data/inequality_dataset_llama3_1-70b-ResearcherDefinition.csv', index_col = 0)
df2 = pandas.read_csv('../data/inequality_dataset_llama3-70b-ResearcherDefinition.csv', index_col = 0)
df3 = pandas.read_csv('../data/inequality_dataset_gemma2-27b-ResearcherDefinition.csv', index_col = 0)
df4 = pandas.read_csv('../data/inequality_dataset_GPT_ResearcherDefinition.csv', index_col=0)

In [44]:
df = pandas.DataFrame()
df['llama31'] = df1['gen_code']
df['llama3'] = df2['gen_code']
df['gemma'] = df3['gen_code']
df['gpt'] = df4['gen_code']

labels = [int(df1['code'][i] != 5) for i in range(len(df1))]

df['code'] = labels

df.head()

,llama31,llama3,gemma,gpt,code
0,1,1,1,1,0
1,1,1,1,1,1
2,1,1,1,1,1
3,0,0,0,0,0
4,0,0,0,0,0


# Metrics without GPT

In [45]:
data = []
for idx, row in df.iterrows():
    data.append(("a1", idx, row["llama31"]))
    data.append(("a2", idx, row["llama3"]))
    data.append(("a3", idx, row["gemma"]))
data[0]

('a1', 0, 1)

In [46]:
atask = agreement.AnnotationTask(data=data)

In [47]:
print("Fleiss's Kappa:", atask.multi_kappa())

Fleiss's Kappa: 0.6788913730892423


## Metrics for ensemble method

In [48]:
ensemble = []

for idx, row in df[['llama31', 'llama3', 'gemma']].iterrows():
    #print(list(row))
    ensemble.append((max(set(list(row)), key=list(row).count)))

df['ensemble'] = ensemble

In [50]:
y_true = df.code
y_pred = df.ensemble

# calculate the precision and recall scores
precision = precision_score(y_true, y_pred, pos_label=0)
recall = recall_score(y_true, y_pred, pos_label=0)
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=0)
cm = confusion_matrix(y_true, y_pred)


conf_matrix = confusion_matrix(y_true, y_pred)
# Extract true positives for each class
true_positives = np.diag(conf_matrix)

# Calculate precision for each class
precision_per_class = precision_score(y_true, y_pred, average=None)

# Calculate proportion of true positives for each class
total_true_positives = np.sum(true_positives)
proportions = true_positives / total_true_positives

weighted_avg_precision = np.sum(precision_per_class * proportions)


print("\nWeighted average metrics:\n")

precision_per_class = precision_score(y_true, y_pred, average=None)
print(f"Precision for each class: {[f'{r:.2f}' for r in precision_per_class]}")
weighted_avg_precision = np.sum(precision_per_class * proportions)
print(f"Weighted Average Precision {weighted_avg_precision:.2f}")
recall_per_class = recall_score(y_true, y_pred, average=None)
print(f"Recall for each class: {[f'{r:.2f}' for r in recall_per_class]}")
weighted_avg_recall = np.sum(recall_per_class * proportions)
print(f"Weighted Average Recall: {weighted_avg_recall:.2f}")
# Calculate F1 score for each class
f1_per_class = f1_score(y_true, y_pred, average=None)
print(f"F1 Score for each class: {[f'{f:.2f}' for f in f1_per_class]}")
# Calculate weighted average F1 score
weighted_avg_f1 = np.sum(f1_per_class * proportions)
print(f"Weighted Average F1 Score: {weighted_avg_f1:.2f}")


Weighted average metrics:

Precision for each class: ['0.67', '0.87']
Weighted Average Precision 0.79
Recall for each class: ['0.80', '0.77']
Weighted Average Recall: 0.78
F1 Score for each class: ['0.73', '0.82']
Weighted Average F1 Score: 0.78


In [51]:
df_agree = df[(df['llama31'] == df['llama3']) & (df['llama31'] == df['gemma']) & (df['llama3'] == df['gemma'])]
y_true = df_agree.code.astype(int)
y_pred = df_agree.gemma.astype(int)

# calculate the precision and recall scores
precision = precision_score(y_true, y_pred, pos_label=0)
recall = recall_score(y_true, y_pred, pos_label=0)
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=0)
cm = confusion_matrix(y_true, y_pred)


conf_matrix = confusion_matrix(y_true, y_pred)
# Extract true positives for each class
true_positives = np.diag(conf_matrix)

# Calculate precision for each class
precision_per_class = precision_score(y_true, y_pred, average=None)

# Calculate proportion of true positives for each class
total_true_positives = np.sum(true_positives)
proportions = true_positives / total_true_positives

weighted_avg_precision = np.sum(precision_per_class * proportions)

    
print("\nWeighted average metrics:\n")

precision_per_class = precision_score(y_true, y_pred, average=None)
print(f"Precision for each class: {[f'{r:.2f}' for r in precision_per_class]}")
weighted_avg_precision = np.sum(precision_per_class * proportions)
print(f"Weighted Average Precision {weighted_avg_precision:.2f}")
recall_per_class = recall_score(y_true, y_pred, average=None)
print(f"Recall for each class: {[f'{r:.2f}' for r in recall_per_class]}")
weighted_avg_recall = np.sum(recall_per_class * proportions)
print(f"Weighted Average Recall: {weighted_avg_recall:.2f}")
# Calculate F1 score for each class
f1_per_class = f1_score(y_true, y_pred, average=None)
print(f"F1 Score for each class: {[f'{f:.2f}' for f in f1_per_class]}")
# Calculate weighted average F1 score
weighted_avg_f1 = np.sum(f1_per_class * proportions)
print(f"Weighted Average F1 Score: {weighted_avg_f1:.2f}")

print(f"N = {len(df_agree)}")
print(f"%agreement = {len(df_agree) / len(df)}")


Weighted average metrics:

Precision for each class: ['0.81', '0.89']
Weighted Average Precision 0.86
Recall for each class: ['0.84', '0.87']
Weighted Average Recall: 0.86
F1 Score for each class: ['0.82', '0.88']
Weighted Average F1 Score: 0.86
N = 952
%agreement = 0.7597765363128491


# Metrics with GPT

Note these are not reported in the paper.

In [52]:
data = []
for idx, row in df.iterrows():
    data.append(("a1", idx, row["llama31"]))
    data.append(("a2", idx, row["llama3"]))
    data.append(("a3", idx, row["gemma"]))
    data.append(("a4", idx, row["gpt"]))
data[0]

('a1', 0, 1)

In [53]:
atask = agreement.AnnotationTask(data=data)

In [54]:
print("Fleiss's Kappa:", atask.multi_kappa())

Fleiss's Kappa: 0.6900216546489316


## Metrics for ensemble method

In [55]:
ensemble = []

for idx, row in df[['llama31', 'llama3', 'gemma', 'gpt']].iterrows():
    #print(list(row))
    ensemble.append((max(set(list(row)), key=list(row).count)))

df['ensemble'] = ensemble

In [56]:
y_true = df.code
y_pred = df.ensemble

# calculate the precision and recall scores
precision = precision_score(y_true, y_pred, pos_label=0)
recall = recall_score(y_true, y_pred, pos_label=0)
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=0)
cm = confusion_matrix(y_true, y_pred)


conf_matrix = confusion_matrix(y_true, y_pred)
# Extract true positives for each class
true_positives = np.diag(conf_matrix)

# Calculate precision for each class
precision_per_class = precision_score(y_true, y_pred, average=None)

# Calculate proportion of true positives for each class
total_true_positives = np.sum(true_positives)
proportions = true_positives / total_true_positives

weighted_avg_precision = np.sum(precision_per_class * proportions)


print("\nWeighted average metrics:\n")

precision_per_class = precision_score(y_true, y_pred, average=None)
print(f"Precision for each class: {[f'{r:.2f}' for r in precision_per_class]}")
weighted_avg_precision = np.sum(precision_per_class * proportions)
print(f"Weighted Average Precision {weighted_avg_precision:.2f}")
recall_per_class = recall_score(y_true, y_pred, average=None)
print(f"Recall for each class: {[f'{r:.2f}' for r in recall_per_class]}")
weighted_avg_recall = np.sum(recall_per_class * proportions)
print(f"Weighted Average Recall: {weighted_avg_recall:.2f}")
# Calculate F1 score for each class
f1_per_class = f1_score(y_true, y_pred, average=None)
print(f"F1 Score for each class: {[f'{f:.2f}' for f in f1_per_class]}")
# Calculate weighted average F1 score
weighted_avg_f1 = np.sum(f1_per_class * proportions)
print(f"Weighted Average F1 Score: {weighted_avg_f1:.2f}")


Weighted average metrics:

Precision for each class: ['0.67', '0.87']
Weighted Average Precision 0.79
Recall for each class: ['0.81', '0.76']
Weighted Average Recall: 0.78
F1 Score for each class: ['0.73', '0.81']
Weighted Average F1 Score: 0.78


In [57]:
df_agree = df[(df['llama31'] == df['llama3']) & (df['llama31'] == df['gemma']) & (df['llama3'] == df['gemma'])]
y_true = df_agree.code.astype(int)
y_pred = df_agree.gemma.astype(int)

# calculate the precision and recall scores
precision = precision_score(y_true, y_pred, pos_label=0)
recall = recall_score(y_true, y_pred, pos_label=0)
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label=0)
cm = confusion_matrix(y_true, y_pred)


conf_matrix = confusion_matrix(y_true, y_pred)
# Extract true positives for each class
true_positives = np.diag(conf_matrix)

# Calculate precision for each class
precision_per_class = precision_score(y_true, y_pred, average=None)

# Calculate proportion of true positives for each class
total_true_positives = np.sum(true_positives)
proportions = true_positives / total_true_positives

weighted_avg_precision = np.sum(precision_per_class * proportions)

    
print("\nWeighted average metrics:\n")

precision_per_class = precision_score(y_true, y_pred, average=None)
print(f"Precision for each class: {[f'{r:.2f}' for r in precision_per_class]}")
weighted_avg_precision = np.sum(precision_per_class * proportions)
print(f"Weighted Average Precision {weighted_avg_precision:.2f}")
recall_per_class = recall_score(y_true, y_pred, average=None)
print(f"Recall for each class: {[f'{r:.2f}' for r in recall_per_class]}")
weighted_avg_recall = np.sum(recall_per_class * proportions)
print(f"Weighted Average Recall: {weighted_avg_recall:.2f}")
# Calculate F1 score for each class
f1_per_class = f1_score(y_true, y_pred, average=None)
print(f"F1 Score for each class: {[f'{f:.2f}' for f in f1_per_class]}")
# Calculate weighted average F1 score
weighted_avg_f1 = np.sum(f1_per_class * proportions)
print(f"Weighted Average F1 Score: {weighted_avg_f1:.2f}")

print(f"N = {len(df_agree)}")
print(f"%agreement = {len(df_agree) / len(df)}")


Weighted average metrics:

Precision for each class: ['0.81', '0.89']
Weighted Average Precision 0.86
Recall for each class: ['0.84', '0.87']
Weighted Average Recall: 0.86
F1 Score for each class: ['0.82', '0.88']
Weighted Average F1 Score: 0.86
N = 952
%agreement = 0.7597765363128491
